Notebook authors: Özgün Haznedar, Elena Gronskaya

This notebook is for plotting of high-resolution sentinel and low-resolution 
landsat image pairs that have been downloaded from Google Earth Engine as TIF 
files. Different scaling is applied to sentinel and landsat pixel values to 
equalize image brightness and adjust for the different data ranges of the 
original files. For non-filtered data, cloud and missing-data pixel filters can 
be applied to only plot high-quality images. 

In [ ]:
import gdal
gdal.UseExceptions()
import matplotlib.pyplot as plt
import numpy as np
import os

from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# WRITE DIRECTORY NAME AND RUN THE NEXT CELL

directory = 'PATH TO DATASET HERE'


In [ ]:

# iterate over files in
# that directory
l8_list = list()
s2_list = list()
for filename in os.listdir(directory):
    f = os.path.join(directory, filename)
    # checking if it is a file
    if os.path.isfile(f) and "L8" in f:
      l8_list.append(f)
    elif os.path.isfile(f) and "S2" in f:
      s2_list.append(f)

l8_list.sort()
s2_list.sort()
file_list = list(zip(l8_list,s2_list))

#Function that converts downloaded files to RGB arrays and corrects exposure of images by scaling
def img_array_RGB(filepath , model):  
  geotiff = gdal.Open(filepath)
  geotiff_arr = geotiff.ReadAsArray()
  geotiff_arr = geotiff_arr.astype(int)
  rgb_image = np.rollaxis(geotiff_arr,0,3)
     
  if model == "sentinel":
    rgb_image = ((rgb_image *255/3558)*1.4).astype(int) # sentinel conversion , *1.4 is manual correction
  elif model == "landsat" :
    rgb_image = ((rgb_image*0.0000275-0.2)*255*4).astype(int) # landsat conversion original , *4 is manual correction
    #rgb_image = ((rgb_image*0.0000275-0.2)*255).astype(int) # landsat conversion original

  else:
    print("NO MODEL SPECIFIED")

  return rgb_image


image_RGBs = list()

for landsat_file , sentinel_file in file_list:

  landsat_rgb = img_array_RGB(landsat_file,"landsat")  
  sentinel_rgb = img_array_RGB(sentinel_file,"sentinel") 

  # Thresholds for clouds and missing data

  low_threshold = 0   
  low_percentile = 5 
  high_threshold = 200
  high_percentile = 99

  if np.percentile(landsat_rgb[:,0,0],low_percentile)> low_threshold and \
     np.percentile(landsat_rgb[0,:,0],low_percentile)> low_threshold and \
     np.percentile(landsat_rgb[0,0,:],low_percentile)> low_threshold and \
     np.percentile(sentinel_rgb[:,0,0],low_percentile)> low_threshold and \
     np.percentile(sentinel_rgb[0,:,0],low_percentile)> low_threshold and \
     np.percentile(sentinel_rgb[0,0,:],low_percentile)> low_threshold and \
     np.percentile(landsat_rgb[:,0,0],high_percentile)< high_threshold and \
     np.percentile(landsat_rgb[0,:,0],high_percentile)< high_threshold and \
     np.percentile(landsat_rgb[0,0,:],high_percentile)< high_threshold and \
     np.percentile(sentinel_rgb[:,0,0],high_percentile)< high_threshold and \
     np.percentile(sentinel_rgb[0,:,0],high_percentile)< high_threshold and \
     np.percentile(sentinel_rgb[0,0,:],high_percentile)< high_threshold:
     filtered_image_RGBs.append([[landsat_rgb,landsat_file],[sentinel_rgb,sentinel_file]])  


In [ ]:
#Plot the images pairwise
for idx , image in enumerate(image_RGBs):
  fig, ax = plt.subplots(1,2,figsize=(20,20))

  ax[0].imshow(image[0][0])
  ax[0].set_title(f"{idx} - {image[0][1][-44:]}")
  #ax[0].set_title(f"Landsat -  90th percintile RGB values : {np.percentile(image[0][:,0,0],90)} ,{np.percentile(image[0][0,:,0],90)} , {np.percentile(image[0][0,0,:],90)}")
  print(idx)

  ax[1].imshow(image[1][0])
  ax[1].set_title(f"{idx} - {image[1][1][-44:]}")
  #ax[1].set_title(f"Sentinel -  90th percintile RGB values : {np.percentile(image[1][:,0,0],90)} ,{np.percentile(image[1][0,:,0],90)} , {np.percentile(image[1][0,0,:],90)}")
  
  plt.show()
  if idx==50:
    break